# IMDb 영화 리뷰의 감성 분석
* 다대일(many-to-one) 구조의 다층 RNN 구현

### 영화 리뷰 데이터 준비

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [2]:
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz

--2022-02-16 07:33:15--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz [following]
--2022-02-16 07:33:15--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘movie_data.csv.gz’

movie_data.csv.gz   100%[===================>]  25.29M   108MB/s   

In [3]:
import os
import gzip
import shutil

with gzip.open('movie_data.csv.gz', 'rb') as f_in, open('movie_data.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [4]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


데이터프레임 df에는 'review'와 'sentiment' 두 개의 컬럼이 있다.

* 'review': 영화 리뷰 텍스트(입력 특성)
* 'sentimant': 예측하려는 타깃 레이블(0은 부정적인 리뷰, 1은 긍적적인 리뷰)

#### 데이터 전처리
1. 텐서플로 데이터셋 객체를 만들고 훈련, 테스트, 검증 데이텃세으로 나눈다.
2. 훈련 데이터셋에 있는 고유한 단어를 찾는다.
3. 고유한 단어를 고유한 정수로 매핑하고 리뷰 텍스트를 정수(고유 단어의 인덱스) 배열로 인코딩
4. 모델에 입력하기 위해 데이터셋을 미니 배치로 나눈다.

In [5]:
# 1 단계: 데이터셋 만들기
target  = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

# 확인
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


* 훈련 / 검증 / 테스트 분할

전체 데이터셋은 5만 개의 샘플을 담고 있다. 처음 2만 5천개의 샘플은 평가를 위해 떼어 놓는다(홀드아웃 테스트 데이터셋). 그 다음 2만 개의 샘플은 훈련에 사용하고 5천개의 샘플은 검증에 사용한다.

In [6]:
tf.random.set_seed(1)

ds_raw = ds_raw.shuffle(
    50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

* Tokenizer와 Encoder
    * `tfds.deprecated.text.Tokenizer`
    * `tfds.deprecated.text.TokenTextEncoder`
* 시퀀스 인코딩: 각 시퀀스에서 마지막 100개 원소만 유지

신경망의 입력으로 데이터를 준비하기 위해 숫자 값으로 인코딩해야 한다. 이렇게 하기 위해 먼저 훈련 데이터셋에서 고유한 단어(토큰)를 찾는다. 데이터셋을 사용하여 고유한 토큰을 찾을 수 있지만 파이썬의 collections 패키지의 Counter 클래스를 사용하는 것이 더 효율적이다.

Counter 객체(token_counts)를 만들어 고유한 단어의 빈도를 수집한다. 텍스트를 단어(또는 토큰)로 나누려면 tensorflow_datasets 패키지가 제공하는 Tokenizer 클래스를 사용할 수 있다.

In [7]:
# 2단계: 고유 토큰(단어) 찾기
from collections import Counter

try:
    tokenizer = tfds.features.text.Tokenizer()
except AttributeError:
    tokenizer = tfds.deprecated.text.Tokenizer()

token_counts = Counter()

for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)
    
print('어휘 사전 크기:', len(token_counts))

어휘 사전 크기: 87007


각각의 고유 단어를 고유 정수로 매핑.

파이썬 딕셔너리를 사용하여 수동으로 처리할 수 있다. 키는 고유 토큰(단어)이고 키에 매핑된 값은 고유한 정수이다. 하지만 tensorflow_datasets 패키지는 이런 매핑과 전체 데이터셋을 인코딩할 수 있는 TokenTextEncoder 클래스를 제공한다. 먼저 고유한 토큰을 전달하여 TokenTextEncoder 클래스로 encoder 객체를 만든다(token_countrs는 토큰과 횟수를 포함하고 있지만 여기서는 횟수는 필요하지 않다). encoder.encode() 메서드를 호출하여 입력 테스트를 정수 리스트로 변환한다.

In [8]:
# 3단계: 고유 토큰을 정수로 인코딩
try:
    encoder = tfds.features.text.TokenTextEncoder(token_counts)
except AttributeError:
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)

example_str = 'This is an example!'
encoder.encode(example_str)

[232, 9, 270, 1123]

In [9]:
# 3-A단계: 변환을 위한 함수 정의
# 즉시 실행 모드가 활성화된 것처럼 입력 텐서를 다룬다.
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

In [10]:
# 3-B단계: 함수를 TF 연산으로 변환
# 위 함수를 tf.py_function으로 감싸서 map()메서드에서 사용할 수 있는 텐서플로 연산으로 변환
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
                          Tout=(tf.int64, tf.int64))

In [11]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('시퀀스 길이:', example[0].shape)
    
example

시퀀스 길이: (24,)
시퀀스 길이: (179,)
시퀀스 길이: (262,)
시퀀스 길이: (535,)
시퀀스 길이: (130,)


(<tf.Tensor: shape=(130,), dtype=int64, numpy=
 array([  579,  1296,    32,   425,    40,   763,  9267,    65,   280,
          308,     6,   481,   155,   473,     2,     3,   684,     9,
          781,   176,   959,   730,  3917,    67,  9905,    13,   277,
           24,    35,   371, 16368,     6,    14, 17231,    29,   187,
         1651,   489,   503,   480,   143,    32,   270,  5851,  2402,
           13,  3592,  3443,   425,  3313,   256,   257,  1577,   117,
            8,   698,   270,   564,    56,     8,    42,  7517,  2629,
          820,    25,    60,    79,   343,    32,   645,    14,   528,
          241,    32,  1980,     8,    56,     8,    42,  1364,   573,
         5183,    43,    12,  3870,    32,   312,   642,   251,  1401,
        17232,     8,   698,   257,   750,     2,     9,    76,   235,
            8,    42,   235,   840,   666,   258, 17233,   419,    32,
        17234,   585,   420,   840,    25,    40,    13,    14,   198,
          266,   623,   173,  

단어 시퀀스를 정수 시퀀스로 변환한 결과 시퀀스 길이가 다르다.

일반적으로 RNN은 다른 길이의 시퀀스를 다룰 수 있지만 미니 배치에 있는 시퀀스는 효율적으로 텐서에 저장하기 위해 동일한 길이가 되어야한다.

크기가 다른 원소를 가진 데이터셋을 미니 배치로 나누기 위해 텐서플로의 batch() 대신 padded_batch() 메서드를 사용한다. 이 메서드는 하나의 배치에 포함되는 모든 원소를 자동으로 0으로 패딩하여 배치에 있는 모든 시퀀스가 동일한 길이가 되도록 만든다.

In [12]:
# 일부 데이터를 추출하여 확인
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('개별 샘플 크기:', example[0].shape)

개별 샘플 크기: (119,)
개별 샘플 크기: (688,)
개별 샘플 크기: (308,)
개별 샘플 크기: (204,)
개별 샘플 크기: (326,)
개별 샘플 크기: (240,)
개별 샘플 크기: (127,)
개별 샘플 크기: (453,)


In [13]:
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], []))
for b in ds_batched:
    print('배치 차원:', b[0].shape)

배치 차원: (4, 688)
배치 차원: (4, 453)


첫 번째 배치의 열의 개수 shape[1]는 688이다. 처음 네 개의 샘플((199,), (688,), (308,), (204,))이 하나의 배치가 되었고 이 샘플 중 가장 큰 크기를 사용.


In [14]:
# 세 개의 데이터셋을 모두 배치 크기 32의 미치 배치로 나눈다
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))

#### 문장 인코딩을 위한 임베딩 층
특성 임베딩은 필수적인 것은 아니지만 단어 벡터의 차원을 줄여주기 때문에 매우 권장되는 전처리 단계이다.

동일한 길이의 시퀀스를 생성하였다. 이 시퀀스의 원소는 고유한 단어의 인덱스에 해당하는 정수이다. 이런 단어 인텍스를 입력 특성으로 변환하는 몇가지 방법이 있다. 

* 원-핫 인코딩

**원-핫 인코딩**을 적용하여 인덱스를 0 또는 1로 이루어진 벡터로 변환할 수 있다. 각 단어는 전체 데이터셋의 고유한 단어의 수에 해당하는 크기를 가진 벡터로 변환된다. 고유한 단어의 수(어휘 사전의 크기)가 $10^4 - 10^5$ 될 수 있어 너무 많은 특성에서 훈련된 모델은 *차원의 저주**로 인한 영향을 받는다. 또 하나를 제외하고 나머지 모든 원소가 0이라 특성 벡터가 매우 희소해진다.

* 고정된 벡터로 변환

각 단어를 (정수가 아닌) 실수 값을 가진 고정된 길이의 벡터로 변환하는 것이다. 원-핫 인코딩과 달리 고정된 길이의 벡터를 사용하여 무한히 많은 실수를 표현할수 있다. (이론적으로 [-1, 1] 사이에서 무한히 실수를 뽑을 수 있다)

* 임베딩
    * 데이터셋에 있는 단어를 표현하는 데 중요한 특성을 자동으로 학습
    * 고유한 단어의 수를 $n_{words}$라고 할 때 고유한 단어의 수보다 훨씬 작게 ($embedding_dim << n_{words}$) 임베딩 벡터(또는 임베딩 차원) 크기를 선택하여 전체 어휘를 입력 특성으로 나타낸다
    * 원-핫 인코딩에 비해 임베딩의 장점
        * 특성 공간의 차원이 축소되므로 차원의 저주로 인한 영향을 감소시킨다.
        * 신경망에서 임베딩 층이 최적화(학습)되기 때문에 중요한 특성이 추출된다.
        

In [15]:
from IPython.display import Image
Image(url='https://git.io/JLdV0', width=700)

* `input_dim`: 단어 개수, 즉 최대 정수 인덱스 + 1(n+2)
* `output_dim`: 임베딩 특성의 크기, 임베딩 벡터의 출력 차원
* `input_length`: (패딩된) 시퀀스 길이
    * 예를 들어, `'This is an example' -> [0, 0, 0, 0, 0, 0, 3, 1, 8, 9]`   
    => input_lenght는 10
* 이 층을 호출 할 때 입력으로 정수값을 받는다. 임베딩 층이 정수를 `[input_dim]` 크기의 실수 벡터로 변환한다.
    * 입력 크기가 `[BATCH_SIZE]`이면 출력 크기는 `[BATCH_SIZE, output_dim]`이 된다.
    * 입력 크기가 `[BATCH_SIZE, 10]`이면 출력 크기는 `[BATCH_SIZE, 10, output_dim]`가 된다.
    

In [16]:
from tensorflow.keras.layers import Embedding

model = tf.keras.Sequential()
model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name='embed-layer'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


`input_length` 매개변수는 필수가 아니다. 하지만 Embedding 층 다음에 Flatten 층이나 Dense 층을 두려면 `input_length` 매개변수를 꼭 지정해야 한다. 또한 입력 시퀀스 길이가 가변적인 경우 `None`으로 지정할 수 있다.

#### RNN 모델 만들기

케라스 Sequntial 클래스를 사용하여 임베딩 층, RNN 층, 완전 연결 층을 연결한다.

* 순환 층에서 사용할 수 있는 클래스
    * SimpleRNN: 완전 연결 순환 층인 기본 RNN
        * `tf.keras.layers.SimpleRNN(units, return_seqeunces=False)`
    * LSTM: 긴 의존성을 감지할 수 있는 LSTM RNN
        * `tf.keras.layers.LSTM(..)`
    * GRU: LSTM의 대안인 GRU 유닛을 사용한 순환 층
        * `tf.keras.layer.GRU(..)`

* `return_sequences` 결정
    * 다층 RNN이면 마지막을 층을 제외하고 모든 RNN층을 `return_sequences=True`로 지정
    * 마지막 RNN층은 문제의 종류에 따라 결정
        * 다대다: `return_sequences=True`
        * 다대일: `return_sequences=False`

In [17]:
# SimpleRNN 층으로 RNN 모델
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 32)) # input_dim=100, output_dim=32
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 32)          2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


In [18]:
# LSTM 층으로 RNN 모델
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [19]:
# GRU 층으로 RNN 모델
from tensorflow.keras.layers import GRU

model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          320000    
                                                                 
 gru (GRU)                   (None, None, 32)          6336      
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 332,705
Trainable params: 332,705
Non-trainable params: 0
_________________________________________________________________


#### 감성 분석 작업을 위한 RNN 모델 만들기

시퀀스 길이가 길기 때문에 길게 미치는 영향을 감지하기 위해 LSTM 층을 사용. 또한, Bidirectional 래퍼 클래스로 LSTM 층을 감싼다. 이 층은 입력 시퀀스를 처음부터 끝까지 그리고 끝에서 처음까지 양방향으로 순환 층을 통과하도록 한다.

In [20]:
embedding_dim = 20
vocab_size = len(token_counts) + 2
tf.random.set_seed(1)

# 모델
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim = vocab_size,
        output_dim = embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='latm-layer'),
        name='bidir-latm'),
    
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 bidir-latm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 컴파일과 훈련
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10)

# 테스트 데이터에서 평가
test_results = bi_lstm_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(test_results[1]*100))

Epoch 1/10
625/625 [==============================] - 165s 252ms/step - loss: 0.5026 - accuracy: 0.7423 - val_loss: 0.5280 - val_accuracy: 0.7474
Epoch 2/10
625/625 [==============================] - 150s 239ms/step - loss: 0.2629 - accuracy: 0.8990 - val_loss: 0.3659 - val_accuracy: 0.8588
Epoch 3/10
625/625 [==============================] - 152s 243ms/step - loss: 0.1697 - accuracy: 0.9420 - val_loss: 0.4489 - val_accuracy: 0.8344
Epoch 4/10
625/625 [==============================] - 149s 239ms/step - loss: 0.0854 - accuracy: 0.9729 - val_loss: 0.4767 - val_accuracy: 0.8496
Epoch 5/10
625/625 [==============================] - 151s 241ms/step - loss: 0.0570 - accuracy: 0.9819 - val_loss: 0.5658 - val_accuracy: 0.8432
Epoch 6/10
625/625 [==============================] - 153s 245ms/step - loss: 0.0529 - accuracy: 0.9837 - val_loss: 0.5469 - val_accuracy: 0.8358
Epoch 7/10
625/625 [==============================] - 150s 239ms/step - loss: 0.0454 - accuracy: 0.9872 - val_loss: 0.6919 -

In [22]:
if not os.path.exists('models'):
    os.mkdir('models')


bi_lstm_model.save('models/Bidir-LSTM-full-length-seq.h5')

* 양방향(bidirectional) RNN
    * Bidirectional 래퍼 클래스는 입력 시퀀스를 정방향과 역방향으로 두 번 처리한다.
    * 기본적으로 정방향과 양방향으로 처리한 결과는 연결된다.
    * 이 방식을 바꾸려면 merge_mode를 'sum', 'mul', 'ave'(평균), 'concat'(기본값) 또는 None으로 지정하여 두 텐서를 리스트로 변환할 수 있다.

SimpleRNN과 같은 다른 종류의 순환 층을 사용할 수도 있다. 일반적인 순환 층으로 만든 모델은 (훈련 데이터에서도) 좋은 예측 성능을 달성하지 못한다. 예를 들어 양방향 LSTM 층을 단방향 SimpleRNN 층으로 바꾸고 전체 길이를 사용한 시퀀스로 모델을 훈련하면 훈련하는 동안 손실이 전혀 감소하지 않는다. 이 데이터셋에 있는 시퀀스가 너무 길기 때문이다. SimpleRNN 층을 사용한 모델은 장기간 의존성을 학습할 수 없고 그레이디언트 감소나 폭주로 인한 영향을 받는다.

SimpleRNN으로 이 데이터셋에서 납득할 수 있는 수준의 예측 성능을 얻기 위해서는 시퀀스 길이를 줄여야한다. '도메인 지식'을 사용하면 영화 리뷰의 마지막 문장에 감성에 관한 정보가 많이 담겨 있다고 가정할 수 있다. 따라서 각 리뷰의 마지막 부분에만 초점을 맞춘다. 

preprocess_datasets()라는 헬퍼 함수를 만들어 전처리 단계 2~4를 연결한다. 이 함수의 `매개변수는 각 리뷰에는 얼마나 많은 토큰을 사용할지 결정하는 max_seq_length`이다. 예를 들어 max_seq_length=100으로 지정하면 100개 이상의 토큰을 가진 리뷰에서 마지막 100개의 토큰만 사용된다. max_seq_length=None으로 지정하면 이전처럼 전체 길이의 시퀀스가 사용된다.

In [23]:
from collections import Counter
def preprocess_datasets(
    ds_raw_train, 
    ds_raw_valid, 
    ds_raw_test,
    max_seq_length=None,
    batch_size=32):

    # 단계 1: 데이터셋 만들기 이미 완료
    # 단계 2: 고유 토큰 찾기
    try:
        tokenizer = tfds.features.text.Tokenizer()
    except AttributeError:
        tokenizer = tfds.deprecated.text.Tokenizer()

    token_counts = Counter()

    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens = tokens[-max_seq_length:]
        token_counts.update(tokens)
    
    print('어휘 사전 크기:', len(token_counts))

    # 단계 3: 텍스트 인코딩
    try:
        encoder = tfds.features.text.TokenTextEncoder(token_counts)
    except AttributeError:
        encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_seq_length is not None:
            encoded_text = encoded_text[-max_seq_length:]
        return encoded_text, label

    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))
    
    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)

    ## 단계 4: 배치 데이터 만들기
    train_data = ds_train.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    valid_data = ds_valid.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    test_data = ds_test.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    return (train_data, valid_data, 
            test_data, len(token_counts))

In [24]:
def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):

    tf.random.set_seed(1)

    # 모델 생성
    model = tf.keras.Sequential()
    
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
            
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
            
        model.add(recurrent_layer)

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [25]:
from tensorflow.keras.layers import Bidirectional

In [26]:
# SimpleRNN 층 하나를 가진 모델을 최대 길이가 토큰 100개인 시퀀스로 훈련
batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

어휘 사전 크기: 58063
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [27]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

Epoch 1/10
625/625 [==============================] - 212s 335ms/step - loss: 0.6980 - accuracy: 0.5061 - val_loss: 0.6835 - val_accuracy: 0.5446
Epoch 2/10
625/625 [==============================] - 212s 339ms/step - loss: 0.5596 - accuracy: 0.6932 - val_loss: 0.4372 - val_accuracy: 0.8078
Epoch 3/10
625/625 [==============================] - 212s 339ms/step - loss: 0.3368 - accuracy: 0.8554 - val_loss: 0.5490 - val_accuracy: 0.8072
Epoch 4/10
625/625 [==============================] - 208s 333ms/step - loss: 0.2349 - accuracy: 0.9088 - val_loss: 0.4910 - val_accuracy: 0.8066
Epoch 5/10
625/625 [==============================] - 211s 338ms/step - loss: 0.1528 - accuracy: 0.9451 - val_loss: 0.6207 - val_accuracy: 0.7300
Epoch 6/10
625/625 [==============================] - 214s 342ms/step - loss: 0.1133 - accuracy: 0.9629 - val_loss: 0.7179 - val_accuracy: 0.6950
Epoch 7/10
625/625 [==============================] - 219s 350ms/step - loss: 0.0752 - accuracy: 0.9740 - val_loss: 0.6476 -

In [28]:
results = rnn_model.evaluate(test_data)

782/782 [==============================] - 47s 60ms/step - loss: 0.8086 - accuracy: 0.6313


In [29]:
print('테스트 정확도: {:.2f}%'.format(results[1]*100))

테스트 정확도: 63.13%


In [38]:
batch_size = 32
embedding_dim = 20
max_seq_length = 1000

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='LSTM', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

results = rnn_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(results[1]*100))

어휘 사전 크기: 86935
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1738740   
                                                                 
 bidir-lstm-layer-0 (Bidirec  (None, 128)              43520     
 tional)                                                         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,790,581
Trainable params: 1,790,581
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 161s 252ms/step - loss: 0.6577 - accurac

In [39]:
batch_size = 32
embedding_dim = 20
max_seq_length = 1000

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='GRU', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

results = rnn_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(results[1]*100))

어휘 사전 크기: 86935
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1738740   
                                                                 
 bidir-gru-layer-0 (Bidirect  (None, 128)              33024     
 ional)                                                          
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,780,085
Trainable params: 1,780,085
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 178s 278ms/step - loss: 0.5234 - accurac

### 전체 길이를 사용한 시퀀스에 단방향 SimpleRNN 적용

In [30]:
batch_size = 32
embedding_dim = 20
max_seq_length = None

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test,
    max_seq_length=max_seq_length,
    batch_size=batch_size
)

vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN',
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=False)

rnn_model.summary()

어휘 사전 크기: 87007
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 simprnn-layer-0 (SimpleRNN)  (None, 64)               5440      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,749,845
Trainable params: 1,749,845
Non-trainable params: 0
_________________________________________________________________


In [31]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

Epoch 1/10
625/625 [==============================] - 778s 1s/step - loss: 0.6989 - accuracy: 0.5012 - val_loss: 0.6990 - val_accuracy: 0.5040
Epoch 2/10
625/625 [==============================] - 772s 1s/step - loss: 0.7008 - accuracy: 0.5030 - val_loss: 0.7002 - val_accuracy: 0.4974
Epoch 3/10
625/625 [==============================] - 772s 1s/step - loss: 0.6967 - accuracy: 0.4931 - val_loss: 0.6931 - val_accuracy: 0.5112
Epoch 4/10
625/625 [==============================] - 767s 1s/step - loss: 0.6946 - accuracy: 0.4990 - val_loss: 0.6929 - val_accuracy: 0.5126
Epoch 5/10
625/625 [==============================] - 774s 1s/step - loss: 0.6942 - accuracy: 0.5010 - val_loss: 0.6929 - val_accuracy: 0.5130
Epoch 6/10
625/625 [==============================] - 759s 1s/step - loss: 0.6938 - accuracy: 0.5044 - val_loss: 0.6930 - val_accuracy: 0.5150
Epoch 7/10
625/625 [==============================] - 765s 1s/step - loss: 0.6935 - accuracy: 0.5070 - val_loss: 0.6930 - val_accuracy: 0.5156

### 데이터셋 만드는 방법: tensorflow_dataset

In [32]:
imdb_bldr = tfds.builder('imdb_reviews')
print(imdb_bldr.info)

imdb_bldr.download_and_prepare()

datasets = imdb_bldr.as_dataset(shuffle_files=False)

datasets.keys()

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2BT0BV/imdb_reviews-train.tfrecord


...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2BT0BV/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


dict_keys(['test', 'train', 'unsupervised'])

In [33]:
imdb_train = datasets['train']
imdb_test = datasets['test']

### Tokenizer와 Encoder
 * `tfds.deprecated.text.Tokenizer`: https://www.tensorflow.org/datasets/api_docs/python/tfds/deprecated/text/Tokenizer
 * `tfds.deprecated.text.TokenTextEncoder`: https://www.tensorflow.org/datasets/api_docs/python/tfds/deprecated/text/TokenTextEncoder

In [34]:
vocab_set = {'a', 'b', 'c', 'd'}
encoder = tfds.deprecated.text.TokenTextEncoder(vocab_set)
print(encoder)

print(encoder.encode(b'a b c d, , : .'))

print(encoder.encode(b'a b c d e f g h i z'))

<TokenTextEncoder vocab_size=6>
[3, 2, 1, 4]
[3, 2, 1, 4, 5, 5, 5, 5, 5, 5]


### 케라스 텍스트 전처리

In [35]:
TOP_K = 200
MAX_LEN = 10

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=TOP_K)

tokenizer.fit_on_texts(['this is an example', 'je suis en forme '])
sequences = tokenizer.texts_to_sequences(['this is an example', 'je suis en forme '])
print(sequences)

tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN)

[[1, 2, 3, 4], [5, 6, 7, 8]]


array([[0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 5, 6, 7, 8]], dtype=int32)

In [36]:
TOP_K = 20000
MAX_LEN = 500

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=TOP_K)

tokenizer.fit_on_texts(
    [example['text'].numpy().decode('utf-8') 
     for example in imdb_train])

x_train = tokenizer.texts_to_sequences(
    [example['text'].numpy().decode('utf-8')
     for example in imdb_train])

print(len(x_train))


x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=MAX_LEN)

print(x_train_padded.shape)

25000
(25000, 500)


### 임베딩

In [37]:
from tensorflow.keras.layers import Embedding

tf.random.set_seed(1)
embed = Embedding(input_dim=100, output_dim=4)

inp_arr = np.array([1, 98, 5, 6, 67, 45])
tf.print(embed(inp_arr))
tf.print(embed(inp_arr).shape)

tf.print(embed(np.array([1])))

[[-0.0208060984 0.0142502077 0.0475785471 -0.00649005175]
 [-0.00420691818 -0.0375086069 -0.00477621704 0.00311584398]
 [0.028728161 -0.0440448038 -0.0428906195 -0.019158531]
 [-0.0248817336 0.0408470519 -0.00285203382 -0.0257614851]
 [0.0443614833 0.00331580639 0.043055404 -0.011118304]
 [-0.0281324144 0.00720113516 0.0192188732 -0.0186921246]]
TensorShape([6, 4])
[[-0.0208060984 0.0142502077 0.0475785471 -0.00649005175]]
